# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
cities_df = pd.read_csv('output_data/cities.csv') # using temp file. Use actual file that was saved in weatherpy
cities_df.head()

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,bluff,-46.6000,168.3333,48.99,77,96,7.00,1619308766
1,1,castro,-24.7911,-50.0119,59.99,74,75,2.01,1619308882
2,2,consdorf,49.7731,6.3383,42.80,65,0,14.97,1619309248
3,3,beitbridge,-22.2167,30.0000,66.78,35,0,4.68,1619309248
4,4,grand gaube,-20.0064,57.6608,75.99,84,100,7.00,1619309248


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
locations = cities_df[["Lat", "Lon"]]
humidity = cities_df["Humidity"].astype(float)
max_humidity = humidity.max()
max_humidity

100.0

In [16]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
perfect_weather_df = cities_df.loc[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 70) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)]
perfect_weather_df.dropna(how='any')
perfect_weather_df


,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date
23,23,port hedland,-20.3167,118.5667,77.00,29,0,9.22,1619309251
73,73,aragarcas,-15.8975,-52.2508,73.40,78,0,1.52,1619309263
128,128,beroroha,-21.6667,45.1667,71.89,68,0,4.14,1619309273
189,189,hervey bay,-25.2986,152.8535,75.00,66,0,4.00,1619309044
349,349,dalbandin,28.8947,64.4101,74.80,10,0,4.81,1619309329
354,354,maceio,-9.6658,-35.7353,75.20,94,0,4.61,1619309031
370,370,oussouye,12.4850,-16.5469,73.40,88,0,4.61,1619309335
408,408,sainte-rose,16.3324,-61.6979,78.80,73,0,5.75,1619309345
436,436,riyadh,24.6877,46.7219,78.80,21,0,6.17,1619308832
480,480,amalapuram,16.5833,82.0167,77.67,88,0,1.97,1619309364


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
# set up additional columns to hold information
hotel_df = perfect_weather_df.loc[:,["City", "Lat", "Lon"]]
hotel_df['Hotel'] = ""
hotel_df

,City,Lat,Lon,Hotel
23,port hedland,-20.3167,118.5667,
73,aragarcas,-15.8975,-52.2508,
128,beroroha,-21.6667,45.1667,
189,hervey bay,-25.2986,152.8535,
349,dalbandin,28.8947,64.4101,
354,maceio,-9.6658,-35.7353,
370,oussouye,12.4850,-16.5469,
408,sainte-rose,16.3324,-61.6979,
436,riyadh,24.6877,46.7219,
480,amalapuram,16.5833,82.0167,


In [27]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lon = row["Lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [28]:
hotel_df

,City,Lat,Lon,Hotel
23,port hedland,-20.3167,118.5667,The Esplanade Hotel
73,aragarcas,-15.8975,-52.2508,Odara Araguaia
128,beroroha,-21.6667,45.1667,
189,hervey bay,-25.2986,152.8535,Oaks Hervey Bay Resort and Spa
349,dalbandin,28.8947,64.4101,Al-Dawood Hotel & Restaurant
354,maceio,-9.6658,-35.7353,Meridiano Hotel
370,oussouye,12.4850,-16.5469,Campement ALULUM
408,sainte-rose,16.3324,-61.6979,villa bagatelle Location saisonnière avec pisc...
436,riyadh,24.6877,46.7219,Hyatt Regency Riyadh Olaya
480,amalapuram,16.5833,82.0167,ROYAL PALACE


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dl>
<dt>Name</dt><dd>{Hotel}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [32]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…